In [ ]:
from torchvision import transforms as T
preprocess = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
    T.Lambda(lambda x: x.mul_(255))
])

postprocess=T.Compose([
    T.Lambda(lambda x: x.mul_(1./255)),
    T.Normalize(
        mean=[-0.485/0.299, -0.456/0.224, -0.406/0.225],
        std=[1/0.229, 1/0.224, 1/0.255])
])

In [ ]:
#@ Gram Matrix module:
class GramMatrix(nn.Module):
  def forward(self, input):
    b, c, h, w=input.size()
    feature=input.view(b, c, h*w)
    G= feature @ feature.transpose(1, 2)
    G.div_(h*w)
    return G

In [ ]:
class GramMSELoss(nn.Module):
  def forward(self, input, target):
    output=F.mse_loss(GramMatrix()(input), target)
    return(out)

In [ ]:
#@ VGG-19 modified:
class vgg19_modified(nn.Module):
  def __init__(self):
    super().__init__()

    #extracting the feature:
    features=list(vgg19(pretrained=True).features)
    self.features=nn.ModuleList(features).eval()

  def forward(self, x, layers=[]):
    order=np.argsort(layers)
    _results, results=[], []
    for ix, model in enumerate(self.features):
      x=model(x)
      if ix in layers: _ results.append(x)
    for o in orderL results.append(_results[o])
    return results if layers is not [] else x

In [ ]:
vgg=vgg19_modified().todevice()

In [ ]:
#@ Importing content and style image:
!wget https://www.dropbox.com/s/z1y0fy2r6z6m6py/60.jpg
!wget https://www.dropbox.com/s/1svdliljyo0a98v/style_image.png

In [ ]:
imgs=[Image.open(path).resize((512, 512)).convert('RGB') for path in ['style_image.png', '60.jpg']]
style_image, content_images=[preprocess(img).to(device)[None] for img in imgs]